In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import AutoModel

In [ ]:
train_data = pd.read_csv("/kaggle/input/emnlp20230-task1/train.csv",encoding='utf8')
train_data.head(20)

In [ ]:
val_data = pd.read_csv("/kaggle/input/emnlp20230-task1/dev.csv",encoding='utf8')
val_data.head(20)

In [ ]:
# Direct Violence 	2
# Passive Violence	1
# Non-Violence	0

## Splitting Data into (Violent and Non-Violent)---->Model1

In [ ]:
# Violence	1
# Non-Violence	0

In [ ]:
def split_main_data(label):
    return (1 if label>0 else 0)

In [ ]:
train_data_splitted = train_data
train_data_splitted.label = train_data.label.apply(split_main_data)
train_data_splitted.head(20)

In [ ]:
val_data_splitted = val_data
val_data_splitted.label = val_data.label.apply(split_main_data)
val_data_splitted.head(20)

In [ ]:
train_data_splitted.to_csv("train_data_splitted.csv",index=False)
val_data_splitted.to_csv("val_data_splitted.csv",index=False)

## Model1 [Violence(1) Vs. Non-Violence(0)]

In [ ]:
!pip install --upgrade pandas

In [ ]:
!pip install --upgrade datasets

In [ ]:
from datasets import load_dataset

In [ ]:
load_train_data_splitted = load_dataset('csv',data_files="/kaggle/working/train_data_splitted.csv")

In [ ]:
load_train_data_splitted

In [ ]:
load_val_data_splitted = load_dataset('csv',data_files="/kaggle/working/val_data_splitted.csv")
load_val_data_splitted

In [ ]:
splitted_data = load_train_data_splitted

In [ ]:
splitted_data['validation'] = load_val_data_splitted['train']

In [ ]:
splitted_data

In [ ]:
from transformers import AutoTokenizer
model_name = "csebuetnlp/banglabert"
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(batch):
    return tokenizer1(batch['text'], padding="max_length", max_length=230, truncation=True)

In [ ]:
splitted_data_encoded  = splitted_data.map(tokenize,batched=True,batch_size=16)

In [ ]:
splitted_data_encoded

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_labels=2
model1 = (AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=num_labels).to(device))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels,preds)
    return {"Accuracy":acc,"Macro F1 Score": f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
def model_init(trial):
    return model1

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(splitted_data_encoded["train"])//batch_size
finetuned_model_name1 = f"{model_name}-VITD-m1opt"
training_args1 = TrainingArguments(output_dir = finetuned_model_name1,
#                                     report_to=None,
                                  num_train_epochs=8,
                                  learning_rate=5e-05,
                                   seed=42,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
#                                   weight_decay=0.01,
                                  evaluation_strategy="epoch",
#                                    save_strategy = "epoch",
                                  disable_tqdm = False,
#                                    load_best_model_at_end = True,
                                  logging_steps = logging_steps,
                                  push_to_hub=True,
                                  log_level="error",
#                                    label_names=["Non-Violence","Voilence"]
                                  )

In [ ]:
from transformers import trainer

trainer1 = Trainer(model=model1,
                  args=training_args1,
                  compute_metrics=compute_metrics,
                  train_dataset = splitted_data_encoded['train'],
                  eval_dataset = splitted_data_encoded['validation'],
                  tokenizer = tokenizer1,
#                   data_collator=data_collator
                  )

In [ ]:
trainer1.train()

In [ ]:
# trainer1.save_model(output_dir="/kaggle/working/best_model_xlm_r")

In [ ]:
trainer1.push_to_hub()

In [ ]:
preds_output = trainer1.predict(splitted_data_encoded['validation'])
preds_output.metrics

## Model2 [Direct-Violence(1) Vs. Passive-Violence(0)]

In [ ]:
data = pd.read_csv("/kaggle/input/emnlp20230-task1/train.csv",encoding='utf8')

In [ ]:
data.head(20)

In [ ]:
violent_train_data = data[data.label!=0]

In [ ]:
violent_train_data.head()

In [ ]:
violent_train_data["label"] = violent_train_data['label'].apply(lambda x: 1 if x==2 else 0) 

In [ ]:
violent_train_data.head()

In [ ]:
data_val = pd.read_csv("/kaggle/input/emnlp20230-task1/dev.csv",encoding='utf8')
data_val.head(20)

In [ ]:
len(data_val[data_val.label!=0])

In [ ]:
violent_val_data = data_val[data_val.label!=0]
violent_val_data.head(20)

In [ ]:
violent_val_data["label"] = violent_val_data['label'].apply(lambda x: 1 if x==2 else 0) 

In [ ]:
violent_val_data.head(20)

In [ ]:
violent_train_data.to_csv("violent_train_data.csv",index=False)
violent_val_data.to_csv("violent_val_data.csv",index=False)

In [ ]:
from datasets import load_dataset

In [ ]:
load_violent_train_data = load_dataset('csv',data_files="/kaggle/working/violent_train_data.csv")
load_violent_val_data = load_dataset('csv',data_files="/kaggle/working/violent_val_data.csv")

In [ ]:
violent_data = load_violent_train_data
violent_data['validation'] = load_violent_val_data['train']

In [ ]:
violent_data

In [ ]:
from transformers import AutoTokenizer
model_name = "xlm-roberta-base"
tokenizer2 = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer2(batch['text'], padding="max_length",max_length=230, truncation=True)

violent_data_encoded  = violent_data.map(tokenize,batched=True,batch_size=8)

In [ ]:
violent_data_encoded

In [ ]:
unique_train_labels = violent_data_encoded['train'].unique('label')
unique_validation_labels = violent_data_encoded['validation'].unique('label')

print("Unique labels in train split:", unique_train_labels)
# print("Unique labels in validation split:", unique_validation_labels)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_labels=2
model2 = (AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=num_labels).to(device))

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels,preds)
    return {"Accuracy":acc,"Macro F1 Score": f1}

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 16
logging_steps = len(violent_data_encoded["train"])//batch_size
finetuned_model_name2 = f"{model_name}-VITD-m2"
training_args2 = TrainingArguments(report_to=None,
                                    output_dir = finetuned_model_name2,
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                   save_strategy = "epoch",
                                  disable_tqdm = False,
                                  logging_steps =logging_steps,
                                   load_best_model_at_end = True,
                                  push_to_hub=True,
                                   label_names = ["Passive Violence", "Direct Violence"]
                                  log_level="error",
                                  )

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
from transformers import trainer

trainer2 = Trainer(model=model2,
                  args=training_args2,
                  compute_metrics=compute_metrics,
                  train_dataset = violent_data_encoded['train'],
                  eval_dataset = violent_data_encoded['validation'],
                  tokenizer = tokenizer2,
#                   data_collator=data_collator
                  )

In [ ]:
trainer2.train()

In [ ]:
trainer2.push_to_hub()

In [ ]:
preds_output = trainer2.predict(violent_data_encoded['validation'])
preds_output.metrics

In [ ]:
preds_output